# Electromagnetism

*Running Sage Math and sagemanifolds with jupyter notebooks tip*: First, I've found that compiling from the github source the development Sage Math version (see my notes on this under the "Computers" section of my [wordpress blog](https://ernestyalumni.wordpress.com)) works with sagemanifolds and either installing sagemanifolds into the binary that you unpack out (the click, download, double click), "breaks" Sage and so that it doesn't run.  Also, following the Sage Math instructions on their website for building from the source didn't work for me (!!!???).  

As a tip on how to run this jupyter notebook and have sagemanifolds available, you'd want to be in the working directory you desire (e.g. `Propulsion/EM`).  But yet your Sage Math build is somewhere else (e.g. `/home/topolo/Public/sage`).  Do this out of the working directory you're currently working out of:
`/home/topolo/Public/sage/sage -n jupyter`

For the rationale, or the math, and how the math corresponds directly to the Sage Math code here, you're going to want to look at [Gravity_Notes_grande.pdf](https://github.com/ernestyalumni/Gravite/blob/master/LaTeXetpdfs/Gravity_Notes_grande.pdf) in my Gravite repository, and in there, the $\mathbb{R}^3$ section, because I define the charts and atlases for Euclidean space $\mathbb{R}^3$ as a *smooth manifold*.  

In [8]:
M = Manifold(4,'M',r'M')
cart_ch = M.chart('t x y z')
U = M.open_subset('U',coord_def={cart_ch: (cart_ch[1]<0, cart_ch[2]!=0)})
cart_ch_U = cart_ch.restrict(U)
sph_ch = U.chart(r'tsp:(-oo,oo):t_{sp} rh:(0,+oo):\rho th:(0,pi):\theta ph:(0,2*pi):\phi')
tsph, rh,th,ph = [sph_ch[i[0]] for i in M.index_generator(1)]
transit_sph_to_cart = sph_ch.transition_map(cart_ch_U, 
                                            [tsph,rh*sin(th)*cos(ph),rh*sin(th)*sin(ph),rh*cos(th)])
Sphnorm = sqrt(sum([cart_ch_U[i]**2 for i in range(1,4)]))
transit_sph_to_cart.set_inverse( cart_ch[0], Sphnorm, 
                                atan2(sqrt( sum([ cart_ch_U[i]**2 for i in range(1,3)])),cart_ch_U[3]),
                                                            atan2(cart_ch_U[2],cart_ch_U[1]))
cyl_ch = U.chart(r'tcy:(-oo,oo):t_{cy} r:(0,+oo) phi:(0,2*pi):\phi zc')
tcy, r,phi,zc = [cyl_ch[i[0]] for i in M.index_generator(1)]
transit_cyl_to_cart = cyl_ch.transition_map(cart_ch_U, [tcy,r*cos(phi),r*sin(phi),zc])
transit_cyl_to_cart.set_inverse(cart_ch_U[0], sqrt(cart_ch_U[1]**2+cart_ch_U[2]**2), 
                                    atan2( cart_ch_U[2],cart_ch_U[1]), cart_ch_U[3])

Note the mostly positive (-+++) convention I use for the Minkowski metric.

In [9]:
g = M.riemannian_metric('g')
g[0,0] = -1 
for i in range(1,4): g[i,i] = 1

### Electric Field

In [23]:
def make_E(ch):
    """
    make_E = make_E(ch)
    make_E creates a time-INDEPENDENT electric field as a 1-form
    
    INPUT/PARAMETER
    ch = sagemanifold chart
    """
    Ecomplst = []
    for i in range(1,4):
        Earglst = ['E'+str(i),] + list(ch[1:])
        Ecomplst.append( function(Earglst[0])(*Earglst[1:]) )
    Ecomplst = [0,]+Ecomplst
    E = ch.domain().diff_form(1)
    E[ch.frame(),:,ch] = Ecomplst
    return E

def make_Et(ch):
    """
    make_Et = make_Et(ch)
    make_Et creates a time-DEPENDENT electric field as a 1-form
    
    INPUT/PARAMETER
    ch = sagemanifold chart
    """
    Ecomplst = []
    for i in range(1,4):
        Earglst = ['E'+str(i),] + list(ch[:])
        Ecomplst.append( function(Earglst[0])(*Earglst[1:]) )
    Ecomplst = [0,]+Ecomplst
    E = ch.domain().diff_form(1)
    E[ch.frame(),:,ch] = Ecomplst
    return E

Examples of using `make_E`, `make_Et` and displaying the results

In [29]:
print make_E(cart_ch).display()
make_Et(sph_ch).display(sph_ch.frame(),sph_ch)

E1(x, y, z) dx + E2(x, y, z) dy + E3(x, y, z) dz


E1(tsp, rh, th, ph) drh + E2(tsp, rh, th, ph) dth + E3(tsp, rh, th, ph) dph

### Magnetic Field

Programming note: `make_B` and `make_Bt` 

In [47]:
import itertools
def make_B(ch):
    """
    make_B = make_B(ch)
    make_B creates a time-INDEPENDENT magnetic field as a 2-form
    
    INPUT/PARAMETER
    ch = sagemanifold chart
    """
    B = ch.domain().diff_form(2)
    farglst = list(ch[1:]) # function argument list, e.g. (x,y,z)

    B[ch.frame(),1,2,ch] = function('B_12')(*farglst)
    B[ch.frame(),2,3,ch] = function('B_23')(*farglst)
    B[ch.frame(),3,1,ch] = function('B_31')(*farglst)

    return B

def make_Bt(ch):
    """
    make_Bt = make_Bt(ch)
    make_Bt creates a time-DEPENDENT electric field as a 2-form
    
    INPUT/PARAMETER
    ch = sagemanifold chart
    """
    B = ch.domain().diff_form(2)
    farglst = list(ch[:]) # function argument list, e.g. (x,y,z)

    B[ch.frame(),1,2,ch] = function('B_12')(*farglst)
    B[ch.frame(),2,3,ch] = function('B_23')(*farglst)
    B[ch.frame(),3,1,ch] = function('B_31')(*farglst)

    return B

In [46]:
print make_Bt(cart_ch).display()
make_B(cyl_ch).display(cyl_ch.frame(),cyl_ch)

B_12(t, x, y, z) dx/\dy - B_31(t, x, y, z) dx/\dz + B_23(t, x, y, z) dy/\dz


B_12(r, phi, zc) dr/\dphi - B_31(r, phi, zc) dr/\dzc + B_23(r, phi, zc) dphi/\dzc

Notice that the orientation is correct (with the right hand rule).

## Electromagnetic field 2-form

In [54]:
EM_F = make_Bt(cart_ch) + make_Et(cart_ch).wedge(cart_ch.coframe()[0] )
EM_F.display()

-E1(t, x, y, z) dt/\dx - E2(t, x, y, z) dt/\dy - E3(t, x, y, z) dt/\dz + B_12(t, x, y, z) dx/\dy - B_31(t, x, y, z) dx/\dz + B_23(t, x, y, z) dy/\dz

In [55]:
EM_F[:]

[                0   -E1(t, x, y, z)   -E2(t, x, y, z)   -E3(t, x, y, z)]
[   E1(t, x, y, z)                 0  B_12(t, x, y, z) -B_31(t, x, y, z)]
[   E2(t, x, y, z) -B_12(t, x, y, z)                 0  B_23(t, x, y, z)]
[   E3(t, x, y, z)  B_31(t, x, y, z) -B_23(t, x, y, z)                 0]

In [61]:
latex(EM_F.exterior_der()[:]); # delete the semi-colon ; and you can get the LaTeX code-I suppress it


$ dF = \left[\left[\left[0, 0, 0, 0\right], \left[0, 0, \frac{\partial\,B_{12}}{\partial {t}} - \frac{\partial\,E_{1}}{\partial y} + \frac{\partial\,E_{2}}{\partial x}, -\frac{\partial\,B_{31}}{\partial {t}} - \frac{\partial\,E_{1}}{\partial z} + \frac{\partial\,E_{3}}{\partial x}\right], \left[0, -\frac{\partial\,B_{12}}{\partial {t}} + \frac{\partial\,E_{1}}{\partial y} - \frac{\partial\,E_{2}}{\partial x}, 0, \frac{\partial\,B_{23}}{\partial {t}} - \frac{\partial\,E_{2}}{\partial z} + \frac{\partial\,E_{3}}{\partial y}\right], \left[0, \frac{\partial\,B_{31}}{\partial {t}} + \frac{\partial\,E_{1}}{\partial z} - \frac{\partial\,E_{3}}{\partial x}, -\frac{\partial\,B_{23}}{\partial {t}} + \frac{\partial\,E_{2}}{\partial z} - \frac{\partial\,E_{3}}{\partial y}, 0\right]\right], \left[\left[0, 0, -\frac{\partial\,B_{12}}{\partial {t}} + \frac{\partial\,E_{1}}{\partial y} - \frac{\partial\,E_{2}}{\partial x}, \frac{\partial\,B_{31}}{\partial {t}} + \frac{\partial\,E_{1}}{\partial z} - \frac{\partial\,E_{3}}{\partial x}\right], \left[0, 0, 0, 0\right], \left[\frac{\partial\,B_{12}}{\partial {t}} - \frac{\partial\,E_{1}}{\partial y} + \frac{\partial\,E_{2}}{\partial x}, 0, 0, \frac{\partial\,B_{12}}{\partial z} + \frac{\partial\,B_{23}}{\partial x} + \frac{\partial\,B_{31}}{\partial y}\right], \left[-\frac{\partial\,B_{31}}{\partial {t}} - \frac{\partial\,E_{1}}{\partial z} + \frac{\partial\,E_{3}}{\partial x}, 0, -\frac{\partial\,B_{12}}{\partial z} - \frac{\partial\,B_{23}}{\partial x} - \frac{\partial\,B_{31}}{\partial y}, 0\right]\right], \left[\left[0, \frac{\partial\,B_{12}}{\partial {t}} - \frac{\partial\,E_{1}}{\partial y} + \frac{\partial\,E_{2}}{\partial x}, 0, -\frac{\partial\,B_{23}}{\partial {t}} + \frac{\partial\,E_{2}}{\partial z} - \frac{\partial\,E_{3}}{\partial y}\right], \left[-\frac{\partial\,B_{12}}{\partial {t}} + \frac{\partial\,E_{1}}{\partial y} - \frac{\partial\,E_{2}}{\partial x}, 0, 0, -\frac{\partial\,B_{12}}{\partial z} - \frac{\partial\,B_{23}}{\partial x} - \frac{\partial\,B_{31}}{\partial y}\right], \left[0, 0, 0, 0\right], \left[\frac{\partial\,B_{23}}{\partial {t}} - \frac{\partial\,E_{2}}{\partial z} + \frac{\partial\,E_{3}}{\partial y}, \frac{\partial\,B_{12}}{\partial z} + \frac{\partial\,B_{23}}{\partial x} + \frac{\partial\,B_{31}}{\partial y}, 0, 0\right]\right], \left[\left[0, -\frac{\partial\,B_{31}}{\partial {t}} - \frac{\partial\,E_{1}}{\partial z} + \frac{\partial\,E_{3}}{\partial x}, \frac{\partial\,B_{23}}{\partial {t}} - \frac{\partial\,E_{2}}{\partial z} + \frac{\partial\,E_{3}}{\partial y}, 0\right], \left[\frac{\partial\,B_{31}}{\partial {t}} + \frac{\partial\,E_{1}}{\partial z} - \frac{\partial\,E_{3}}{\partial x}, 0, \frac{\partial\,B_{12}}{\partial z} + \frac{\partial\,B_{23}}{\partial x} + \frac{\partial\,B_{31}}{\partial y}, 0\right], \left[-\frac{\partial\,B_{23}}{\partial {t}} + \frac{\partial\,E_{2}}{\partial z} - \frac{\partial\,E_{3}}{\partial y}, -\frac{\partial\,B_{12}}{\partial z} - \frac{\partial\,B_{23}}{\partial x} - \frac{\partial\,B_{31}}{\partial y}, 0, 0\right], \left[0, 0, 0, 0\right]\right]\right] = 0  $


In [66]:
EM_F.hodge_dual(g)[:]

[                  0  I*B_23(t, x, y, z)  I*B_31(t, x, y, z)  I*B_12(t, x, y, z)]
[-I*B_23(t, x, y, z)                   0    I*E3(t, x, y, z)   -I*E2(t, x, y, z)]
[-I*B_31(t, x, y, z)   -I*E3(t, x, y, z)                   0    I*E1(t, x, y, z)]
[-I*B_12(t, x, y, z)    I*E2(t, x, y, z)   -I*E1(t, x, y, z)                   0]

In [67]:
g[:]

[-1  0  0  0]
[ 0  1  0  0]
[ 0  0  1  0]
[ 0  0  0  1]

$d*F$ (for $d*F = 4 \pi * J$)

In [71]:
EM_F.hodge_dual(g).exterior_der()[:]

[[[0, 0, 0, 0],
  [0,
   0,
   I*d(B_23)/dy - I*d(B_31)/dx + I*d(E3)/dt,
   -I*d(B_12)/dx + I*d(B_23)/dz - I*d(E2)/dt],
  [0,
   -I*d(B_23)/dy + I*d(B_31)/dx - I*d(E3)/dt,
   0,
   -I*d(B_12)/dy + I*d(B_31)/dz + I*d(E1)/dt],
  [0,
   I*d(B_12)/dx - I*d(B_23)/dz + I*d(E2)/dt,
   I*d(B_12)/dy - I*d(B_31)/dz - I*d(E1)/dt,
   0]],
 [[0,
   0,
   -I*d(B_23)/dy + I*d(B_31)/dx - I*d(E3)/dt,
   I*d(B_12)/dx - I*d(B_23)/dz + I*d(E2)/dt],
  [0, 0, 0, 0],
  [I*d(B_23)/dy - I*d(B_31)/dx + I*d(E3)/dt,
   0,
   0,
   I*d(E1)/dx + I*d(E2)/dy + I*d(E3)/dz],
  [-I*d(B_12)/dx + I*d(B_23)/dz - I*d(E2)/dt,
   0,
   -I*d(E1)/dx - I*d(E2)/dy - I*d(E3)/dz,
   0]],
 [[0,
   I*d(B_23)/dy - I*d(B_31)/dx + I*d(E3)/dt,
   0,
   I*d(B_12)/dy - I*d(B_31)/dz - I*d(E1)/dt],
  [-I*d(B_23)/dy + I*d(B_31)/dx - I*d(E3)/dt,
   0,
   0,
   -I*d(E1)/dx - I*d(E2)/dy - I*d(E3)/dz],
  [0, 0, 0, 0],
  [-I*d(B_12)/dy + I*d(B_31)/dz + I*d(E1)/dt,
   I*d(E1)/dx + I*d(E2)/dy + I*d(E3)/dz,
   0,
   0]],
 [[0,
   -I*d(B_12)/dx + I*d(

In [78]:
latex( EM_F.hodge_dual(g).exterior_der()[:] ); # delete the semi-colon ; and you can get the LaTeX code-I suppress it

$\left[\left[\left[0, 0, 0, 0\right], \left[0, 0, i \, \frac{\partial\,B_{23}}{\partial y} - i \, \frac{\partial\,B_{31}}{\partial x} + i \, \frac{\partial\,E_{3}}{\partial {t}}, -i \, \frac{\partial\,B_{12}}{\partial x} + i \, \frac{\partial\,B_{23}}{\partial z} - i \, \frac{\partial\,E_{2}}{\partial {t}}\right], \left[0, -i \, \frac{\partial\,B_{23}}{\partial y} + i \, \frac{\partial\,B_{31}}{\partial x} - i \, \frac{\partial\,E_{3}}{\partial {t}}, 0, -i \, \frac{\partial\,B_{12}}{\partial y} + i \, \frac{\partial\,B_{31}}{\partial z} + i \, \frac{\partial\,E_{1}}{\partial {t}}\right], \left[0, i \, \frac{\partial\,B_{12}}{\partial x} - i \, \frac{\partial\,B_{23}}{\partial z} + i \, \frac{\partial\,E_{2}}{\partial {t}}, i \, \frac{\partial\,B_{12}}{\partial y} - i \, \frac{\partial\,B_{31}}{\partial z} - i \, \frac{\partial\,E_{1}}{\partial {t}}, 0\right]\right], \left[\left[0, 0, -i \, \frac{\partial\,B_{23}}{\partial y} + i \, \frac{\partial\,B_{31}}{\partial x} - i \, \frac{\partial\,E_{3}}{\partial {t}}, i \, \frac{\partial\,B_{12}}{\partial x} - i \, \frac{\partial\,B_{23}}{\partial z} + i \, \frac{\partial\,E_{2}}{\partial {t}}\right], \left[0, 0, 0, 0\right], \left[i \, \frac{\partial\,B_{23}}{\partial y} - i \, \frac{\partial\,B_{31}}{\partial x} + i \, \frac{\partial\,E_{3}}{\partial {t}}, 0, 0, i \, \frac{\partial\,E_{1}}{\partial x} + i \, \frac{\partial\,E_{2}}{\partial y} + i \, \frac{\partial\,E_{3}}{\partial z}\right], \left[-i \, \frac{\partial\,B_{12}}{\partial x} + i \, \frac{\partial\,B_{23}}{\partial z} - i \, \frac{\partial\,E_{2}}{\partial {t}}, 0, -i \, \frac{\partial\,E_{1}}{\partial x} - i \, \frac{\partial\,E_{2}}{\partial y} - i \, \frac{\partial\,E_{3}}{\partial z}, 0\right]\right], \left[\left[0, i \, \frac{\partial\,B_{23}}{\partial y} - i \, \frac{\partial\,B_{31}}{\partial x} + i \, \frac{\partial\,E_{3}}{\partial {t}}, 0, i \, \frac{\partial\,B_{12}}{\partial y} - i \, \frac{\partial\,B_{31}}{\partial z} - i \, \frac{\partial\,E_{1}}{\partial {t}}\right], \left[-i \, \frac{\partial\,B_{23}}{\partial y} + i \, \frac{\partial\,B_{31}}{\partial x} - i \, \frac{\partial\,E_{3}}{\partial {t}}, 0, 0, -i \, \frac{\partial\,E_{1}}{\partial x} - i \, \frac{\partial\,E_{2}}{\partial y} - i \, \frac{\partial\,E_{3}}{\partial z}\right], \left[0, 0, 0, 0\right], \left[-i \, \frac{\partial\,B_{12}}{\partial y} + i \, \frac{\partial\,B_{31}}{\partial z} + i \, \frac{\partial\,E_{1}}{\partial {t}}, i \, \frac{\partial\,E_{1}}{\partial x} + i \, \frac{\partial\,E_{2}}{\partial y} + i \, \frac{\partial\,E_{3}}{\partial z}, 0, 0\right]\right], \left[\left[0, -i \, \frac{\partial\,B_{12}}{\partial x} + i \, \frac{\partial\,B_{23}}{\partial z} - i \, \frac{\partial\,E_{2}}{\partial {t}}, -i \, \frac{\partial\,B_{12}}{\partial y} + i \, \frac{\partial\,B_{31}}{\partial z} + i \, \frac{\partial\,E_{1}}{\partial {t}}, 0\right], \left[i \, \frac{\partial\,B_{12}}{\partial x} - i \, \frac{\partial\,B_{23}}{\partial z} + i \, \frac{\partial\,E_{2}}{\partial {t}}, 0, i \, \frac{\partial\,E_{1}}{\partial x} + i \, \frac{\partial\,E_{2}}{\partial y} + i \, \frac{\partial\,E_{3}}{\partial z}, 0\right], \left[i \, \frac{\partial\,B_{12}}{\partial y} - i \, \frac{\partial\,B_{31}}{\partial z} - i \, \frac{\partial\,E_{1}}{\partial {t}}, -i \, \frac{\partial\,E_{1}}{\partial x} - i \, \frac{\partial\,E_{2}}{\partial y} - i \, \frac{\partial\,E_{3}}{\partial z}, 0, 0\right], \left[0, 0, 0, 0\right]\right]\right]$

In [76]:
EM_F.hodge_dual(g).exterior_der().hodge_dual(g)[:] 

[d(E1)/dx + d(E2)/dy + d(E3)/dz,
 -d(B_12)/dy + d(B_31)/dz + d(E1)/dt,
 d(B_12)/dx - d(B_23)/dz + d(E2)/dt,
 d(B_23)/dy - d(B_31)/dx + d(E3)/dt]

In [81]:
latex(EM_F.hodge_dual(g).exterior_der().hodge_dual(g)[:]); 
# delete the semi-colon ; and you can get the LaTeX code-I suppress it

\left[\frac{\partial\,E_{1}}{\partial x} + \frac{\partial\,E_{2}}{\partial y} + \frac{\partial\,E_{3}}{\partial z}, -\frac{\partial\,B_{12}}{\partial y} + \frac{\partial\,B_{31}}{\partial z} + \frac{\partial\,E_{1}}{\partial {t}}, \frac{\partial\,B_{12}}{\partial x} - \frac{\partial\,B_{23}}{\partial z} + \frac{\partial\,E_{2}}{\partial {t}}, \frac{\partial\,B_{23}}{\partial y} - \frac{\partial\,B_{31}}{\partial x} + \frac{\partial\,E_{3}}{\partial {t}}\right]

$\left[\frac{\partial\,E_{1}}{\partial x} + \frac{\partial\,E_{2}}{\partial y} + \frac{\partial\,E_{3}}{\partial z}, -\frac{\partial\,B_{12}}{\partial y} + \frac{\partial\,B_{31}}{\partial z} + \frac{\partial\,E_{1}}{\partial {t}}, \frac{\partial\,B_{12}}{\partial x} - \frac{\partial\,B_{23}}{\partial z} + \frac{\partial\,E_{2}}{\partial {t}}, \frac{\partial\,B_{23}}{\partial y} - \frac{\partial\,B_{31}}{\partial x} + \frac{\partial\,E_{3}}{\partial {t}}\right]$

## Current 1-form, Current conservation, and the other side (Right-Hand Side (RHS)) of $d*F$

In [111]:
def make_J(ch):
    """
    make_J = make_J(ch)
    make_J creates a time-INDEPENDENT current as a 1-form
    
    INPUT/PARAMETER
    ch = sagemanifold chart
    """
    Jcomplst = []
    for i in range(1,4):
        Jarglst = ['j'+str(i),] + list(ch[1:])
        Jcomplst.append( function(Jarglst[0])(*Jarglst[1:]) )
    Jcomplst = [-function('rho')(*list(ch[1:])),] +Jcomplst
    J = ch.domain().diff_form(1)
    J[ch.frame(),:,ch] = Jcomplst
    return J


def make_Jt(ch):
    """
    make_Jt = make_Jt(ch)
    make_Jt creates a time-DEPENDENT current as a 1-form
    
    INPUT/PARAMETER
    ch = sagemanifold chart
    """
    Jcomplst = []
    for i in range(1,4):
        Jarglst = ['j'+str(i),] + list(ch[:])
        Jcomplst.append( function(Jarglst[0])(*Jarglst[1:]) )
    Jcomplst = [-function('rho')(*list(ch[:])),]+Jcomplst
    J = ch.domain().diff_form(1)
    J[ch.frame(),:,ch] = Jcomplst
    return J
    

In [112]:
print make_Jt(cart_ch).display() # these are examples of displaying the 4-current as 1-form in 
                                    # Cartesian and cylindrical coordinates
make_Jt(cyl_ch).display(cyl_ch.frame(),cyl_ch)

-rho(t, x, y, z) dt + j1(t, x, y, z) dx + j2(t, x, y, z) dy + j3(t, x, y, z) dz


-rho(tcy, r, phi, zc) dtcy + j1(tcy, r, phi, zc) dr + j2(tcy, r, phi, zc) dphi + j3(tcy, r, phi, zc) dzc

In [113]:
make_Jt(cart_ch).hodge_dual(g).hodge_dual(g).display()

rho(t, x, y, z) dt - j1(t, x, y, z) dx - j2(t, x, y, z) dy - j3(t, x, y, z) dz

So here, I had successfully shown that $*d*F = **J$ or $*d*F = 4\pi ** J$ (in cgs units), thus recovering Gauss's law and Ampere's law.

In [115]:
latex( make_Jt(cart_ch).hodge_dual(g).hodge_dual(g)[:]);
# delete the semi-colon ; and you can get the LaTeX code-I suppress it

$\boxed{ \left[\frac{\partial\,E_{1}}{\partial x} + \frac{\partial\,E_{2}}{\partial y} + \frac{\partial\,E_{3}}{\partial z}, -\frac{\partial\,B_{12}}{\partial y} + \frac{\partial\,B_{31}}{\partial z} + \frac{\partial\,E_{1}}{\partial {t}}, \frac{\partial\,B_{12}}{\partial x} - \frac{\partial\,B_{23}}{\partial z} + \frac{\partial\,E_{2}}{\partial {t}}, \frac{\partial\,B_{23}}{\partial y} - \frac{\partial\,B_{31}}{\partial x} + \frac{\partial\,E_{3}}{\partial {t}}\right] = \left[\rho\left({t}, x, y, z\right), -j_{1}\left({t}, x, y, z\right), -j_{2}\left({t}, x, y, z\right), -j_{3}\left({t}, x, y, z\right)\right] }$

Current conservation is easily calculated, $*d*J=0$:

In [116]:
make_Jt(cart_ch).hodge_dual(g).exterior_der().hodge_dual(g).display(cart_ch)

M --> R
(t, x, y, z) |--> d(j1)/dx + d(j2)/dy + d(j3)/dz + d(rho)/dt

## Lorentz Force

In [117]:
def make_beta(ch):
    """
    make_beta = make_beta(ch)
    make_beta creates a time-INDEPENDENT velocity field
    
    INPUT/PARAMETER
    ch = sagemanifold chart
    """
    betacomplst = []
    for i in range(1,4):
        betaarglst = ['beta'+str(i),] + list(ch[1:])
        betacomplst.append( function(betaarglst[0])(*betaarglst[1:]) )
    betacomplst = [1,]+betacomplst
    beta = ch.domain().vector_field()
    beta[ch.frame(),:,ch] = betacomplst
    return beta

def make_betat(ch):
    """
    make_betat = make_betat(ch)
    make_betat creates a time-DEPENDENT velocity field
    
    INPUT/PARAMETER
    ch = sagemanifold chart
    """
    betacomplst = []
    for i in range(1,4):
        betaarglst = ['beta'+str(i),] + list(ch[:])
        betacomplst.append( function(betaarglst[0])(*betaarglst[1:]) )
    betacomplst = [1,]+betacomplst
    beta = ch.domain().vector_field()
    beta[ch.frame(),:,ch] = betacomplst
    return beta

In [119]:
make_beta(cart_ch).display()

d/dt + beta1(x, y, z) d/dx + beta2(x, y, z) d/dy + beta3(x, y, z) d/dz

For interior products, you're going to have to dig into how sagemanifolds implements Tensor products, tensor contractions, and the use of index notation, as sagemanifolds doesn't have a "stand-alone" interior product function.  From my [EuclideanManifold.py](https://github.com/ernestyalumni/Propulsion/blob/master/Learn/EuclideanManifold.py) implementation in sagemanifolds, look at my curl function (`def curl`) as a template for implementing interior products.  

In [124]:
betaeg = make_betat(cart_ch)
Beg = make_Bt(cart_ch)
(betaeg['^i']*Beg['_ij']).display()

(-B_12(t, x, y, z)*beta2(t, x, y, z) + B_31(t, x, y, z)*beta3(t, x, y, z)) dx + (B_12(t, x, y, z)*beta1(t, x, y, z) - B_23(t, x, y, z)*beta3(t, x, y, z)) dy + (-B_31(t, x, y, z)*beta1(t, x, y, z) + B_23(t, x, y, z)*beta2(t, x, y, z)) dz

So we now have a prescription on how to implement both the **interior product** and the **curl** of 2 "vectors" -  
if you want this:

$-i_{\mathbf{\beta}} B$ which is the differential form version of $\mathbf{\beta} \times B$ (curl), then do this in sagemanifolds:

`-betaeg['^i']*Beg['_ij']`

In [128]:
q = var('q',"real") # define a single charge variable in Sage Math

In [134]:
LorentzForce1form =  make_Et(cart_ch) - make_beta(cart_ch)['^i']*make_Bt(cart_ch)['_ij']

EY : 20160530 I have a question; is there a good way for Sage Math variables such as q in this case (var) to "play with" sagemanifolds tensors?  For instance, I obtain this when I multiply a sagemanifolds 1-form by a Sage Math variable (var) q:

In [135]:
q * LorentzForce1form

TypeError: unsupported operand parent(s) for '*': '<type 'tuple'>' and 'Free module /\^1(M) of 1-forms on the 4-dimensional differentiable manifold M'

In [136]:
5.*LorentzForce1form

1-form on the 4-dimensional differentiable manifold M

Nevertheless, for $q=1$, then the 1-form version of the Lorentz Force $F$ is given by the following, and keep in mind that we integrate 1-forms, we don't integrate vectors (because it goes back to how we transport vectors along a curve, and 1-forms either abscond, circumvent, this problem or is the most natural way to do integration on manifolds): 

In [139]:
latex( LorentzForce1form.display() );

$F = \left( B_{12}\left({t}, x, y, z\right) \beta_{2}\left(x, y, z\right) - B_{31}\left({t}, x, y, z\right) \beta_{3}\left(x, y, z\right) + E_{1}\left({t}, x, y, z\right) \right) \mathrm{d} x + \left( -B_{12}\left({t}, x, y, z\right) \beta_{1}\left(x, y, z\right) + B_{23}\left({t}, x, y, z\right) \beta_{3}\left(x, y, z\right) + E_{2}\left({t}, x, y, z\right) \right) \mathrm{d} y + \left( B_{31}\left({t}, x, y, z\right) \beta_{1}\left(x, y, z\right) - B_{23}\left({t}, x, y, z\right) \beta_{2}\left(x, y, z\right) + E_{3}\left({t}, x, y, z\right) \right) \mathrm{d} z$